In [ ]:
%matplotlib inline
%load_ext autoreload
%autoreload 2
import skimage.io as skio
import skimage
import matplotlib.pyplot as plt
plt.rcParams['figure.dpi'] = 150
import tifffile
import numpy as np
from utils import *
import os
from pathlib import Path
import trackpy as tp

In [ ]:
in_file = r"examples/20240206-114518_WT_aligned_drift_corrected.tif"
out_dir = "./out"

fit_method = "lq"
box_side_length = 5
drift = 0
min_gradient = 600

max_link_displacement_px = 3
min_tray_length = 3

In [ ]:

basename = os.path.basename(in_file)
out_dir = Path(out_dir)
os.makedirs(out_dir, exist_ok=True)

In [ ]:
take_only_walkers_on_fibre_trajectory(in_file, out_dir / basename)
basename_noext, ext = os.path.splitext(basename)

In [ ]:
#os.environ['HDF5_DISABLE_VERSION_CHECK']='0'
#TOOD add out parameter
!python -m picasso localize {out_dir/basename} --fit-method {fit_method} --box-side-length {box_side_length}  --gradient {min_gradient} --drift {drift}



In [ ]:
# Hack -- for now just rename the out file. This is dangerous in multithreaded environment.
out_locs = out_dir / (basename_noext + "_locs.hdf5")
new_suffix = (
    f"__locs_{fit_method}_box{box_side_length}_grad{min_gradient}_drift{drift}.hdf5"
)
new_out_locs = out_dir / (basename_noext + new_suffix)
print(new_out_locs)
out_locs.with_suffix(".yaml").rename(new_out_locs.with_suffix(".yaml"))
out_locs.rename(new_out_locs)

In [ ]:
for max_link_displacement_px in [1]:
    locs = pd.read_hdf(new_out_locs, "locs")
    locs["mass"] = locs.photons

    tray = tp.link(locs, max_link_displacement_px)

    # count the length of trajectories
    tray_by_particle = tray.groupby(["particle"])
    tray["length"] = tray_by_particle["particle"].transform("count")

    # tray.length.hist()
    print("mean drift ", tp.compute_drift(tray).mean())

    # Exclude very short trays
    tray = tray.query(f"length>={min_tray_length}")

    steps = tray.groupby(["particle"]).apply(get_steps_from_df)
    steps["step_len"] = np.sqrt(steps.dx**2 + steps.dy**2)
    print(len(steps))

    suffix = f"__link{max_link_displacement_px}_trajlen{min_tray_length}"
    base_linked_tray = Path(f'{new_out_locs.with_suffix("")}{suffix}')
    tray_out = base_linked_tray.with_suffix(".traj.csv.gz")
    tray.to_csv(tray_out)
    steps_out = base_linked_tray.with_suffix(".steps.csv.gz")
    steps.to_csv(steps_out)

    fig, axes = plt.subplots(nrows=3, ncols=1, figsize=(5, 15))

    plt.sca(axes[0])
    tray.length.hist()
    plt.xlabel("Trajectory Length")
    plt.title("Histogram of trajectory steps")

    plt.sca(axes[1])
    steps.plot.scatter("dx", "dy", ax=axes[1])
    plt.title("Size of steps")

    plt.sca(axes[2])
    import numpy as np
    import scipy
    from scipy.stats import norm
    import matplotlib.pyplot as plt

    step_len_signed = steps.step_len * np.random.choice((-1, 1), size=len(steps))
    # Generate some data for this demonstration.
    data = step_len_signed

    # Fit a normal distribution to the data:
    mu, std = norm.fit(data)
    print("Diff: ", std**2 / 2)

    # Plot the histogram.
    plt.hist(data, bins=15, density=True, alpha=0.6, color="g")

    # Plot the PDF.
    xmin, xmax = plt.xlim()
    x = np.linspace(xmin, xmax, 100)
    p = norm.pdf(x, mu, std)
    plt.plot(x, p, "k", linewidth=2)
    title = "Fit results: mu = %.2f,  std = %.2f" % (mu, std)

    plt.title(title)

    # perform Anderson-Darling Test
    from scipy.stats import anderson

    anderson_data = anderson(data)

    is_normal = anderson_data.statistic < anderson_data.critical_values[1]
    if not is_normal:
        normal_str = f"NOT NORMAL AT {anderson_data.significance_level[1]}%"
    else:
        normal_str = f"normal at {anderson_data.significance_level[1]}%"
    print(normal_str)
    plt.xlabel("Step size (px)\n" + normal_str)
    plt.savefig(base_linked_tray.with_suffix(".png"))

    info = {}
    info["diff_1D_px_px_frame"] = float(get_diff_from_steps(steps))
    print("Diff (in px^2/frame): ", info["diff_1D_px_px_frame"])

    px_to_nm = 110
    frame_to_s = 0.665

    info["diff_1D_nm_nm_s"] = (
        info["diff_1D_px_px_frame"] * px_to_nm * px_to_nm / frame_to_s
    )
    print("Diff (in nm^2/s): ", info["diff_1D_nm_nm_s"])
    info["time_per_1000nm_s"] = 1000 * 1000 / info["diff_1D_nm_nm_s"] / 2
    print("time per 1000nm (s): ", info["time_per_1000nm_s"])

    info["mean_step_nm_s"] = float(
        np.sqrt(2 * info["diff_1D_nm_nm_s"] * frame_to_s) / frame_to_s
    )
    print("mean step (nm/s):", info["mean_step_nm_s"])

    diff_info_out = base_linked_tray.with_suffix(".diff")

    info["normality_str"] = normal_str
    info["anderson_test"] = anderson_data

    write_yaml(info, diff_info_out)

In [ ]:
plt.hist2d(steps.dx, steps.dy, bins=75)

In [ ]:
hist, x_edges, y_edges = np.histogram2d(steps.dx, steps.dy, bins=75)

In [ ]:
from scipy.stats import multivariate_normal 
mean, cov = multivariate_normal.fit(np.array([steps.dx, steps.dy]).T)
normal_dist = multivariate_normal(mean, cov)

In [ ]:
normal_dist.pdf([0,0])

eigen_values, eigen_vectors = np.linalg.eig(cov)

In [ ]:
#dist = (multivariate_normal)

x_centers = (x_edges[:-1] + x_edges[1:]) / 2
y_centers = (y_edges[:-1] + y_edges[1:]) / 2
x_mesh, y_mesh = np.meshgrid(x_centers, y_centers)  # Creates two 2D arrays
xy_combined = np.array([x_mesh.ravel(), y_mesh.ravel()]).T


plt.imshow(hist, extent=[x_edges[0], x_edges[-1], y_edges[0], y_edges[-1]], origin='lower')
plt.contour(x_mesh, y_mesh, normal_dist.pdf(xy_combined).reshape(75, 75), colors='k', alpha=0.5)
plt.quiver(mean[0], mean[1], eigen_vectors[0,0], eigen_vectors[0,1], color='red')
plt.quiver(mean[0], mean[1], eigen_vectors[1,0], eigen_vectors[1,1], color='blue')

In [ ]:
#Top ten longest trajectories
top_ten = list(tray.sort_values(by='length', ascending=False).groupby('particle', sort=False).first().index[0:10])

In [ ]:
def plot_trajectory(ids: list):
    for traj_id in ids:
        one_traj = tray.query(f'particle=={traj_id}')
        one_traj.plot('x', 'y', label=traj_id, ax=plt.gca() )

plot_trajectory(top_ten[0:3])

In [ ]:
VAL = np.diag(eigen_values)
V = eigen_vectors
V_inv = np.linalg.inv(V)
V@VAL@V_inv